***GENERATED CODE FOR bankdepodt PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job', 'transformation_label': 'String Indexer'}], 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '24', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'unemployed', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job'}, {'feature_label': 'job', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'marital', 'transformation_label': 'String Indexer'}], 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'divorced', 'max': 'single', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'marital'}, {'feature_label': 'marital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('marital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'default', 'transformation_label': 'String Indexer'}], 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'default'}, {'feature_label': 'default', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('default')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'housing', 'transformation_label': 'String Indexer'}], 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'housing'}, {'feature_label': 'housing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('housing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan', 'transformation_label': 'String Indexer'}], 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan'}, {'feature_label': 'loan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'contact', 'transformation_label': 'String Indexer'}], 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'contact'}, {'feature_label': 'contact', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('contact')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'month', 'transformation_label': 'String Indexer'}], 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'apr', 'max': 'oct', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'month'}, {'feature_label': 'month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'poutcome', 'transformation_label': 'String Indexer'}], 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'poutcome'}, {'feature_label': 'poutcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('poutcome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'deposit', 'transformation_label': 'String Indexer'}], 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '24', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'deposit'}, {'feature_label': 'deposit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('deposit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    Decision_tree_classifier = DecisionTreeClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    Decision_tree_classifier.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            Decision_tree_classifier.score(X_test, y_test))

    data = {'model': Decision_tree_classifier,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run bankdepodtHooks.ipynb
try:
	#sourcePreExecutionHook()

	bankdepositanalysistl = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Bank_Deposit_Analysis_TL.csv', 'filename': 'Bank_Deposit_Analysis_TL.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://colaberry.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bankdepodtHooks.ipynb
try:
	#transformationPreExecutionHook()

	bankdepofeatureeng = TransformationMain.run(bankdepositanalysistl,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "42.71", "stddev": "13.64", "min": "25", "max": "78", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "job", "transformation_label": "String Indexer"}], "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "admin.", "max": "unemployed", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job"}, {"transformationsData": [{"feature_label": "marital", "transformation_label": "String Indexer"}], "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "divorced", "max": "single", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "marital"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "primary", "max": "unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "default", "transformation_label": "String Indexer"}], "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "default"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "balance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "1405.88", "stddev": "2233.54", "min": "-199", "max": "7944", "missing": "0"}, "updatedLabel": "balance"}, {"transformationsData": [{"feature_label": "housing", "transformation_label": "String Indexer"}], "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "housing"}, {"transformationsData": [{"feature_label": "loan", "transformation_label": "String Indexer"}], "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan"}, {"transformationsData": [{"feature_label": "contact", "transformation_label": "String Indexer"}], "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "contact"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "17.79", "stddev": "8.72", "min": "1", "max": "29", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"feature_label": "month", "transformation_label": "String Indexer"}], "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "apr", "max": "oct", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "232.62", "stddev": "124.99", "min": "89", "max": "618", "missing": "0"}, "updatedLabel": "duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "1.67", "stddev": "1.05", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "campaign"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pdays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "40.92", "stddev": "90.54", "min": "-1", "max": "299", "missing": "0"}, "updatedLabel": "pdays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "24", "mean": "0.79", "stddev": "2.17", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "previous"}, {"transformationsData": [{"feature_label": "poutcome", "transformation_label": "String Indexer"}], "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "poutcome"}, {"transformationsData": [{"feature_label": "deposit", "transformation_label": "String Indexer"}], "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "24", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "deposit"}]}))

	#transformationPostExecutionHook(bankdepofeatureeng)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bankdepodtHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=DecisionTreeClassifier(bankdepofeatureeng, ["age", "balance", "day", "duration", "campaign", "pdays", "previous", "job_stringindexer", "marital_stringindexer", "education_stringindexer", "default_stringindexer", "housing_stringindexer", "loan_stringindexer", "contact_stringindexer", "month_stringindexer", "poutcome_stringindexer"], "deposit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
